##### In this notebook I going to work with sample data from dog breed inentification competiotion (images only of 2 breeds present in sample data) 

In [1]:
import os, sys
#current_dir = os.getcwd()
DATA_HOME_DIR =  '/home/denys/projects/ml-models/kaggle/dog-breed-identification/data/'
#DATA_HOME_DIR =  '/home/denys/projects/ml-models/kaggle/dog-breed-identification/data/sample/'

In [18]:
%matplotlib inline

import numpy as np

from keras.preprocessing import image
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold


import utils; reload(utils)
from utils import plot_learning_curve

In [3]:
labels_df = pd.read_csv("data/labels.csv")

def get_breed(id):
    return labels_df.loc[labels_df["id"] == id]["breed"].values[0]

In [3]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [12]:
target_size=(244,244)

def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical',
                target_size=target_size):
    return gen.flow_from_directory(dirname, target_size=target_size, 
                                   class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

def get_data(path):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None)
    return np.concatenate([batches.next() for i in range(batches.samples)])

In [8]:
train_data = get_data(DATA_HOME_DIR+'train')

Found 9225 images belonging to 120 classes.


In [10]:
save_array(DATA_HOME_DIR+'train_244_244.data', train_data)

In [4]:
train_data = load_array(DATA_HOME_DIR+'train_244_244.data')

In [13]:
batches = get_batches(DATA_HOME_DIR + 'train', shuffle=False, batch_size=1)

Found 9225 images belonging to 120 classes.


In [14]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [19]:
train_classes = batches.classes
train_labels = onehot(train_classes)
train_filenames = batches.filenames
train_class_indices = batches.class_indices

In [20]:
model_path = DATA_HOME_DIR + "models/"

In [10]:
def model():
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(224,224, 3)),
            Conv2D(32,(3,3), activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Conv2D(64,(3,3), activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Conv2D(128,(3,3), activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            BatchNormalization(),
            Dense(2, activation='softmax')
        ])
    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [11]:
def train(model, batches, val_batches, initialisation=False):
    if initialisation:
        model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit_generator(batches, steps_per_epoch=batches.n / batch_size, epochs=10, validation_data=val_batches, 
                    validation_steps=val_batches.n, verbose=1, callbacks=[csv_logger])

    model.optimizer.lr = 0.00001
    model.fit_generator(batches, steps_per_epoch=batches.n / batch_size, epochs=100, validation_data=val_batches, 
                        validation_steps=val_batches.n, verbose=1, callbacks=[csv_logger])

In [21]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('vgg16_0128_2020.log', append=True)

batch_size=10
kf = KFold(n_splits=3, shuffle=True, random_state=1)
kf.get_n_splits(train_data)
print(kf) 

KFold(n_splits=3, random_state=1, shuffle=True)


In [20]:
kf = KFold(n_splits=3, shuffle=True, random_state=1)
kf.get_n_splits(train_data)
print(kf) 

batch_size=10

base_model = model()

init = False
for train_index, valid_index in kf.split(train_data):
    print train_index, valid_index
    train_split, valid_split = train_data[train_index], train_data[valid_index]
    train_labels_split, valid_labels_split = train_labels[train_index], train_labels[valid_index]

    batches = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.2, horizontal_flip=True).flow(train_split, train_labels_split, batch_size=batch_size, shuffle=True)
    val_batches = image.ImageDataGenerator().flow(valid_split, valid_labels_split, batch_size=batch_size, shuffle=False)
    
    train(base_model, batches, val_batches, init)
    init = False

KFold(n_splits=3, random_state=1, shuffle=True)


TypeError: __call__() takes exactly 2 arguments (1 given)

In [24]:
base_model.save_weights(model_path[:-len("models/sample/")] + "models/new_model_sample.weights")

In [ ]:
valid_data, valid_labels, valid_filenames, valid_class_indices 

In [29]:
valid_class_indices

{'doberman': 0, 'pomeranian': 1}

In [17]:
base_model.test_on_batch(valid_data, valid_labels)

[0.21754244, 0.9333334]

Seems that simple model was not able capture all usefull properties of data, 
so I tried to create more complicated model with 3 convolution layers and was right my test accuracy is 0.93 now

Now lets try transfered learning with next models:
    >VGG16
    >ResNet50
    >InceptionV3
    >InceptionResNetV2
    >MobileNet

In [12]:
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet import MobileNet

# = {"vgg16": VGG16(weights='imagenet', include_top=False),
         # "resnet50" : ResNet50(weights='imagenet', include_top=False),
          #"InceptionV3": InceptionV3(weights='imagenet', include_top=False),
          #"InceptionResNetV2": InceptionResNetV2(weights='imagenet', include_top=False)}
 #        }

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

for model_name, base_model in {"vgg16": VGG16(weights='imagenet', include_top=False)}.items():
    print model_name
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    predictions = Dense(120, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', metrics=["accuracy"], loss='categorical_crossentropy')
    model.optimiser = lr=0.00005
    
    model.load_weights(model_path + "/" + model_name + '_0128_1556.h5')
    
    for train_index, valid_index in kf.split(train_data):
        train_split, valid_split = train_data[train_index], train_data[valid_index]
        train_labels_split, valid_labels_split = train_labels[train_index], train_labels[valid_index]

        #rotation_range=15, width_shift_range=0.1, 
                               #height_shift_range=0.1, zoom_range=0.2, horizontal_flip=True
        batches = image.ImageDataGenerator().flow(train_split, train_labels_split, batch_size=batch_size, shuffle=True)
        val_batches = image.ImageDataGenerator().flow(valid_split, valid_labels_split, batch_size=batch_size, shuffle=False)
    
    
        model.fit_generator(batches, steps_per_epoch=batches.n / batch_size, epochs=30, validation_data=val_batches, 
                        validation_steps=val_batches.n, verbose=1, callbacks=[csv_logger])
            
    model.save_weights(model_path + "/" + model_name + '_0128_2020.h5')
   # print model.test_on_batch(valid_data, valid_labels)

vgg16
Epoch 1/30
615/615 [==============================] - 449s 730ms/step - loss: 6.6826 - acc: 0.5580 - val_loss: 6.7118 - val_acc: 0.5516
Epoch 2/30
615/615 [==============================] - 446s 726ms/step - loss: 6.5704 - acc: 0.5750 - val_loss: 6.6197 - val_acc: 0.5616
Epoch 3/30
615/615 [==============================] - 446s 726ms/step - loss: 6.5822 - acc: 0.5750 - val_loss: 6.7159 - val_acc: 0.5621
Epoch 4/30
615/615 [==============================] - 446s 725ms/step - loss: 6.5096 - acc: 0.5834 - val_loss: 6.6363 - val_acc: 0.5700
Epoch 5/30
615/615 [==============================] - 446s 726ms/step - loss: 6.5264 - acc: 0.5807 - val_loss: 6.7379 - val_acc: 0.5511
Epoch 6/30
615/615 [==============================] - 446s 725ms/step - loss: 6.4790 - acc: 0.5873 - val_loss: 6.6878 - val_acc: 0.5517
Epoch 7/30
614/615 [============================>.] - ETA: 0s - loss: 6.4808 - acc: 0.5853

In [ ]:
plot_learning_curve("vgg16_0128_1557.log")

In [ ]:
from keras.layers import Dense, GlobalAveragePooling2D

for model_name, base_model in models.items():
    print model_name
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    predictions = Dense(120, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='rmsprop', metrics=["accuracy"], loss='categorical_crossentropy')
    
    for train_index, valid_index in kf.split(train_data):
        train_split, valid_split = train_data[train_index], train_data[valid_index]
        train_labels_split, valid_labels_split = train_labels[train_index], train_labels[valid_index]

        batches = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.2, horizontal_flip=True).flow(train_split, train_labels_split, batch_size=batch_size, shuffle=True)
        val_batches = image.ImageDataGenerator().flow(valid_split, valid_labels_split, batch_size=batch_size, shuffle=False)
    
    
        model.fit_generator(batches, steps_per_epoch=batches.n / batch_size, epochs=10, validation_data=val_batches, 
                        validation_steps=val_batches.n, verbose=1, callbacks=[csv_logger])
            
    model.save_weights(model_path + "/" + model_name + '.h5')
   # print model.test_on_batch(valid_data, valid_labels)

InceptionResNetV2
Epoch 1/10
615/615 [==============================] - 536s 871ms/step - loss: 2.0528 - acc: 0.5356 - val_loss: 1.3072 - val_acc: 0.7303
Epoch 2/10
615/615 [==============================] - 522s 849ms/step - loss: 1.3119 - acc: 0.6979 - val_loss: 1.5568 - val_acc: 0.7401
Epoch 7/10
615/615 [==============================] - 522s 849ms/step - loss: 1.2715 - acc: 0.7205 - val_loss: 1.5353 - val_acc: 0.7501
Epoch 8/10
615/615 [==============================] - 522s 850ms/step - loss: 1.2840 - acc: 0.7189 - val_loss: 1.6225 - val_acc: 0.7466
Epoch 9/10
615/615 [==============================] - 523s 850ms/step - loss: 1.3214 - acc: 0.7218 - val_loss: 1.6152 - val_acc: 0.7473
Epoch 10/10
615/615 [==============================] - 523s 850ms/step - loss: 1.3093 - acc: 0.7249 - val_loss: 1.6137 - val_acc: 0.7495
Epoch 1/10
615/615 [==============================] - 549s 893ms/step - loss: 1.5033 - acc: 0.7031 - val_loss: 1.4089 - val_acc: 0.8041
Epoch 2/10
615/615 [=========

In [29]:
model.test_on_batch(valid_data, valid_labels)

[0.076204494, 0.9666667]